<a href="https://colab.research.google.com/github/MJSahebnasi/temporal-network-platform/blob/main/thesis_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model**

In [1]:
class Simulation:
  pass